In [1]:
import os
import shutil

def parse_and_update_txt_annot(img_path, txt_path, old_class, new_class, output_txt_path):

    # Đọc file nhãn
    with open(txt_path, "r") as file_label:
        lines = file_label.read().strip().split('\n')
    
    updated_lines = []

    for line in lines:
        if line.strip():  # Kiểm tra dòng không rỗng
            objbud = line.split(' ')
            current_class = int(objbud[0])  # Lớp hiện tại
            
            # Chỉ xử lý các dòng có lớp là old_class
            if current_class == old_class:
                objbud[0] = str(new_class)  # Thay thế lớp
                updated_line = " ".join(objbud)  # Gộp lại dòng đã sửa
                updated_lines.append(updated_line)
    
    # Nếu không còn dòng hợp lệ, trả về False
    if not updated_lines:
        return False
    
    # Ghi dữ liệu đã sửa sang file nhãn mới
    with open(output_txt_path, "w") as output_file:
        output_file.write("\n".join(updated_lines))
    
    return True


def create_paths_list(path):

    full_path = []
    files = sorted(os.listdir(path))
    
    for file in files:
        full_path.append(os.path.join(path, file))
        
    return full_path


def process_and_update_files(img_files_paths, annot_files_paths, old_class, new_class, output_dir):

    img_files = create_paths_list(img_files_paths)
    annot_files = create_paths_list(annot_files_paths)
    
    # Tạo thư mục output
    output_label_dir = os.path.join(output_dir, "labels")
    output_image_dir = os.path.join(output_dir, "images")
    os.makedirs(output_label_dir, exist_ok=True)  # Thư mục nhãn
    os.makedirs(output_image_dir, exist_ok=True)  # Thư mục ảnh
    
    # Kiểm tra ánh xạ ảnh-nhãn
    if len(img_files) != len(annot_files):
        raise ValueError("Số lượng tệp ảnh và nhãn không khớp!")
    
    for img_path, txt_path in zip(img_files, annot_files):
        # Giữ nguyên tên file cho nhãn trong thư mục output
        output_txt_path = os.path.join(output_label_dir, os.path.basename(txt_path))
        output_img_path = os.path.join(output_image_dir, os.path.basename(img_path))
        
        # Cập nhật nhãn và kiểm tra nếu có thay đổi lớp
        has_valid_lines = parse_and_update_txt_annot(img_path, txt_path, old_class, new_class, output_txt_path)
        
        if has_valid_lines:
            # Sao chép ảnh vào thư mục output nếu tệp nhãn có dòng hợp lệ
            shutil.copy(img_path, output_img_path)
        else:
            # Nếu không còn dòng nào hợp lệ, xóa nhãn đã ghi (nếu có)
            if os.path.exists(output_txt_path):
                os.remove(output_txt_path)
                os.remove(output_img_path)

In [5]:
process_and_update_files("jerawat.v4i.yolov11/train/images", "jerawat.v4i.yolov11/train/labels", 4, 5, "whitehead2")